In [1]:
from arcgis import GIS
gis = GIS('pro')

import pandas as pd
import numpy as np

In [2]:
#List all assets in the folder
items = gis.users.me.items(folder='Peru_raw_data')

In [3]:
#Specify X and Y field names
x_fld = "Point_X"
y_fld = "Point_Y"

In [4]:
the_log = open('log.txt','w')
the_log.write('File,Type,Valid,Coerced,Total_rows,Numeric_coords,Valid_coords\n')

63

In [5]:
#Existing items
existing_items = [item.title for item in gis.users.me.items(folder = 'Peru_derived_layers')]

In [7]:
for the_item in items:
    var_name = the_item.name; theStr = f"{var_name},"
    var_type = the_item.type; theStr += f"{var_type},"
    theName = the_item.name.split(".")[0]
    if theName in existing_items:
        print(theStr)
        continue
    
    #Get the data from the item
    if var_type == 'Microsoft Excel':
        df = pd.read_excel(the_item.get_data())
    elif var_type == 'CSV':
        try:
            df = pd.read_csv(the_item.get_data(),encoding='utf-8')
        except:
            theStr+="Invalid encoding\n"
            the_log.write(theStr)
            print(theStr)
            continue
    else: 
        theStr += "Invalid type\n"
        the_log.write(theStr,end='')
        print(theStr)
        continue

    #Search for coordinate fields
    if not(x_fld in df.columns and y_fld in df.columns): 
        theStr += f"Invalid fields\n"
        the_log.write(theStr)
        print(theStr,end='')
        continue

    #Extract just coordinate columns
    df_coords = df.loc[:,[x_fld,y_fld]]

    #Check dtype of coordinate fields, convert if needed
    if df_coords[x_fld].dtype != np.float or df_coords[x_fld].dtype != np.float: 
        theStr += f"Valid,Coerced,"
        #Convert to number
        df_coords[x_fld] = pd.to_numeric(df_coords[x_fld],errors='coerce')
        df_coords[y_fld] = pd.to_numeric(df_coords[y_fld],errors='coerce')
        #Remove bad rows
        df_coords.dropna(how='any',inplace=True)
    else:
        theStr += "Valid,Float,"
        
    #Add the shape to the output
    theStr += f"{df_coords.shape[0]},"

    #Filter for records with valid values
    xy_mask = (df_coords[x_fld]>=-180) & (df_coords[x_fld]<=180) & (df_coords[y_fld]>=-90) & (df_coords[x_fld]<=90)
    df_valid = df_coords.loc[xy_mask].reset_index(drop=True)
    theStr += f",{df_valid.shape[0]}\n"
    the_log.write(theStr)
    print(theStr,end='')

    #Convert to spatial dataframe
    sdf = pd.DataFrame.spatial.from_xy(df_valid,x_column=x_fld,y_column=y_fld)

    #Upload to AGOL
    theName = the_item.name.split(".")[0]
    the_lyr = sdf.spatial.to_featurelayer(title=theName,folder='Peru_derived_layers')
    the_lyr.share(groups=['a81478c63d434e33829d2168c4dfdd93'])

Copy_of_vanGeen_WHO_AppendixB.xlsx,Microsoft Excel,Invalid fields
Complete_Environmental_Sample_Sheet.xlsx,Microsoft Excel,
River_fish_hg_enviro_gps.xlsx,Microsoft Excel,Invalid fields
suspar.xlsx,Microsoft Excel,Invalid fields
Compiled_Study_Sampling_Points3.csv,CSV,Invalid fields
PAM_Inventory_All_2016_V2.csv,CSV,Invalid encoding

FishAnalysisLagCategories.csv,CSV,
Mercury_Complete_Env_Sample_Sheet_LDAS.csv,CSV,
baseline_data_20171109.dta,CSV,
baseline_data_20171116.dta,CSV,
followup_data_20171109.dta,CSV,
followup_data_20171116.dta,CSV,Invalid encoding

ASSET_ENG_031016_CLEANED.DTA,CSV,Invalid fields
economic_data.dta,CSV,Invalid encoding

FINAL_Hunt_follow-up_hair_results_JHG_10-12-16.dta,CSV,Invalid encoding

FISH_CONSUMPTION_ENG_031016_CLEANED_CW.DTA,CSV,Invalid fields
HgBlood_Baseline_20170918.dta,CSV,Invalid encoding

HgBlood_FollowUp_20170918.dta,CSV,Invalid encoding

HH_Mining.dta,CSV,Invalid encoding

HH_ROSTER_ENG_031016_CLEANED.DTA,CSV,Invalid encoding

METADATA_030316_CLE

In [ ]:
the_log.close()

In [ ]:
the_item.get_data()

In [ ]:
df=pd.read_csv('C:\\Users\\jpfay\\AppData\\Local\\Temp\\PAM_Inventory_All_2016_V2.csv')